In [4]:
from ephys_queries import db_setup_core
from dotenv import load_dotenv
import pandas as pd
import numpy as np
from pathlib import Path
from onep.distance import pairwise_correlation
from spiketimes.df.surrogates import shuffled_isi_spiketrains_by
from spiketimes.df.statistics import ifr_by
from onep.stats import p_adjust
from onep.utils import create_combined_col
from scipy import stats
from citalopram_project.load import load_spikes, load_neurons

In [5]:
neurons = load_neurons()
spikes = load_spikes(block_name="pre").merge(neurons[["neuron_id", "session_name", "cluster"]])

In [7]:
from tqdm.notebook import tqdm

def shuffled_isi_correlation_test(
    df_spikes, 
    binwidth=1, 
    n_reps=1000, 
    cell_col="neuron_id", 
    spiketimes_col="spiketimes", 
    t_start=0,
    sigma=0,
    limit=None
):
    fs = 1/binwidth
    df_binned = ifr_by(
        df_spikes,
        spiketimes_col=spiketimes_col,
        spiketrain_col=cell_col,
        sigma=sigma,
        t_start=t_start,
        fs=fs
    )
    df_tidy = pairwise_correlation(
        df_binned,
        cell_col=cell_col,
        value_col="ifr", 
        return_tidy=True,
        fillna=0
    )
    real = df_tidy["value"].values
    n_obs = len(real)
    reps = np.empty((n_obs, n_reps))
    p = np.empty(n_obs)
    for i in tqdm(range(n_reps)):
        df2 = shuffled_isi_spiketrains_by(df_spikes, by_col="neuron_id")
        df_binned = ifr_by(
            df2,
            spiketimes_col=spiketimes_col,
            spiketrain_col=cell_col,
            sigma=sigma,
            t_start=t_start
        )
        if limit is not None:
            bins = np.sort(df_binned["time"].unique())
            max_bin = bins[:limit].max()
            df_binned = df_binned.loc[lambda x: x["time"] < max_bin]
        df_tidy_s = pairwise_correlation(
            df_binned,
            cell_col=cell_col,
            value_col="ifr", 
            return_tidy=True,
            fillna=0
        )
        df_tidy_s = df_tidy_s.rename(columns={"value": "bs_rep"})
        df_tidy_s = pd.merge(df_tidy[["cell_combination", "neuron_1", "neuron_2"]], df_tidy_s, how="left")
        reps[:, i] = df_tidy_s["bs_rep"].values
    for i in range(n_obs):
        p[i] = np.nanmean(np.abs(reps[i]) > np.abs(real[i])) * 2
    p = p_adjust(p)
    return df_tidy.assign(p = p)

In [11]:
session_names = neurons.session_name.unique()
frames = []
for session in tqdm(session_names):
    df1 = spikes.loc[lambda x: x.session_name == session]
    res = shuffled_isi_correlation_test(df1, binwidth=1, n_reps=1000)
    frames.append(res.assign(session_name=session))

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [13]:
correlation_test_results = pd.concat(frames)

In [22]:
from citalopram_project.load import get_data_dir
derived_data_dir = get_data_dir() / "derived"
derived_data_dir.mkdir(exist_ok=True)

# correlation_test_results.to_parquet(derived_data_dir / "pre_1s_spont_correlation_test.parquet.gzip", compression="gzip")

# pd.read_parquet(derived_data_dir / "pre_1s_spont_correlation_test.parquet.gzip")

# correlation_test_results.p < 0.05)

In [12]:
def bin_and_correlate(
    df_spikes,
    binwidth=1,
    spiketimes_col="spiketimes", 
    cell_col="neuron_id",
    t_start=0,
    sigma=0,
    ):
    ...
    df_binned = ifr_by(
        df_spikes,
        spiketimes_col=spiketimes_col,
        spiketrain_col=cell_col,
        sigma=sigma,
        t_start=t_start,
        fs=1/binwidth,
    )
    df_tidy = pairwise_correlation(
        df_binned,
        cell_col=cell_col,
        value_col="ifr", 
        return_tidy=True,
        fillna=0
    )
    return df_tidy

In [15]:
df_list = []
bins = [0.01, 0.1, 0.2, 0.5, 1, 2, 5, 10, 30, 60, 120]
for binwidth in bins:
    for session in tqdm(session_names):
        df1 = spikes.loc[lambda x: x.session_name == session]
        res = bin_and_correlate(df1, binwidth=1)
        df_list.append(res.assign(session_name=session, binwidth=binwidth))

binwidth_res = pd.concat(df_list)


  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

...

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

In [25]:
# binwidth_res.to_parquet(derived_data_dir / "binwidth_spont_correlations.parquet.gzip", compression="gzip")